# Reading comment list for a video

This script can read all the comments and their metadata for a list of TikTok videos.
You have to specify the list of TikTok video ids in the video_ids_file vector (one line for each id).

### Setup

In [49]:
from tikapi import TikAPI, ValidationException, ResponseException
import json
import glob
import os

api = TikAPI("1jDHBpHXtsycrmEqK5AVvhvPZq3N2ZGeFNJKzYEJ2MHTPb4y")
User = api.user(accountKey="6EjyV4OptbvUz1cKgMTnvbXNH1wKPqLVV42jKlzydbVrFLnb")

output_file = "./data/comment_list.json"
video_ids_file = "./data/video_ids.txt"

max_requests_for_each_video = 5
number_of_comment_each_request = 30 #(max possible value is 30)
j = 0 #single video request counter
total_n_comments_read = 0 #total comments counter

### Read video ids from file

In [50]:
#read video ids from file
video_ids = []
with open(video_ids_file) as infile:
    video_ids = [line.rstrip() for line in infile]
print("video ids read from file:",video_ids)


video ids read from file: ['7144023384853171462', '7144246488879713542']


### Run the API request

In [51]:
result = [] #it will contain all the json responses for each request

for video_id in video_ids: #for each video
    print("\n\n#########################################\n\nProcessing video",video_id)
    try:
        response = User.posts.comments.list(
            media_id = video_id,
            count = number_of_comment_each_request # (max possible value is 30)
        )

        response_json = response.json()
        #print(response_json)
        
        j=1
        while(response):    
            #append response data to result list
            result.append(response_json)

            #print comments of response data
            for comment in response_json['comments']:
                comment_text = comment['text']
                print(total_n_comments_read,":",comment_text)
                total_n_comments_read += 1 
                
            if j >= max_requests_for_each_video: #check if we reached maximum n of requests for this video
                break #stop
                
            #if not, do next request
            cursor = response.json().get('cursor')
            print("\nGetting next items, cursor value:", cursor,"\n")
            response = response.next_items()  
            response_json = response.json()
            j+=1

    except ValidationException as e:
        print(e, e.field)

    except ResponseException as e:
        print(e, e.response.status_code)



#########################################

Processing video 7144023384853171462
0 : MI SCUSI MA QUANDO FARÀ LA MARCIA SU ROMA SA NON VORREI PERDERMI L’EVETO… C’È IL RITORNO DELLA SERIE A
1 : "do you know applicare¿" HAHA💀💀
2 : Grande Giorgia! Ti volevo anche chiedere: dici che noi giovani dovremo andare in pensione a 100 anni o per i 90 ci riusciamo?
3 : do you know applicare??
4 : Su questo ha perfettamente ragione e la 194 in pochi l'hanno letta e ne conoscono ogni suo punto .Giusto apicarla nella sua interezza
5 : aiuto
6 : lol
7 : bro
8 : love ❤️
9 : Più chiara di così nn può essere stata👏👏👏e chi nn lo capisce nn conosce l italiano🙈🙈🙈, Giorgia avanti nn ditroeggiare🥰🥰🥰🥰🥰🥰🥰🥰
10 : spero che se la storia si ripete veramente la marcia su roma sarà tipo un evento per la vincita dell’europa league. forza maggica❤️❤️
11 : Non ho capito
12 : quando ci aumentate le paghe le pensioni e vi diminuite i vostri 40.000,00€ al mese ,provate voi a vivere con 1500€ al mese benzina luce gas il cibo

117 : comunque spero veramente che nessuno la voterà veramente
118 : Schizzata🥰
119 : certa gente dovrebbe passare da Amplifon...non ci sentono so sordi..Brava Giorgia forse l'urlo lo sentono certi babbei 🇮🇹🇮🇹🇮🇹👍
120 : brava Giorgia bisogna avere fiducia in te, finalmente ci vuole una donna che governi l'Italia
121 : FORZA CONTEEEE
122 : GIORGIA MELONI PREMIER
123 : I think it's time for action ,,,stop bra bra bra,systemize economic growth in Italy, international business, that means u have be social and not immig
124 : A Fiat 124 e meglio
125 : Mellone
126 : @😡😡😡😡😡😡😡😡😡😡😡😡😡😡😭😭😭😭😭😭
127 : brava Giorgia per le donne abbiamo figli
128 : bravissima Meloni
129 : Giorgia 🇨🇵🇮🇹🇮🇹🇨🇵
130 : sei super in gamba Giorgia ti voterei mille volte
131 : la gente non capisce quello che non vuole capire ma questi sono matti fanno finta di non capire per me e dare i diritti sei grande la gente gli dà
132 : forza giorgia
133 : Grazie Giorgia
134 : bravo GIORGIA
135 : Sono innamorato di questa donna coraggiosa

## Save data to json file

In [52]:
#after appending all responses to result list, dump them into a single json file
print("total number of json files acquired with the API requests:",len(result))
print("total number of comments:",len(total_n_comments_read))


#first check if output file already exist
if os.path.exists(output_file):
    print("output file already exist, i will append the results to it.")
    
    #remove the closing bracket of the current json file (last character of file)
    with open(output_file, 'rb+') as filehandle:
        filehandle.seek(-1, os.SEEK_END)
        filehandle.truncate()
        
    
    final_json = ","
    for i,elem in enumerate(result): #for each json in the list

        #parse json into a string and append it to final_json
        str_elem = json.dumps(elem)
        final_json += str_elem

        if i < len(result)-1:  #if it's not last json, append ','
            final_json+=","

    final_json+="]" #end of file
        
        
    with open(output_file, "a") as outfile: #append final_json to output file
        outfile.write(final_json)
    
    
        
else:
    print("output file does not already exist, i will create it from zero with the results.")

    
    final_json = "["
    for i,elem in enumerate(result): #for each json in the list

        #parse json into a string and append it to final_json
        str_elem = json.dumps(elem)
        final_json += str_elem

        if i < len(result)-1:  #if it's not last json, append ','
            final_json+=","

    final_json+="]" #end of file

    with open(output_file, "w") as outfile: #write final_json to output file
        outfile.write(final_json)


total number of json files acquired with the API requests: 10
output file does not already exist, i will create it from zero with the results.


## Read data from json file

In [53]:
#reading a json file and outputting just the comments

with open(output_file) as infile:
    # returns JSON object as 
    # a dictionary
    data = json.load(infile)

    # Iterating through the json
    # list
    i=0
    for elem in data:
        for comment in elem['comments']:
            comment_text = comment['text']
            print(i,":",comment_text)
            i+=1 

0 : MI SCUSI MA QUANDO FARÀ LA MARCIA SU ROMA SA NON VORREI PERDERMI L’EVETO… C’È IL RITORNO DELLA SERIE A
1 : "do you know applicare¿" HAHA💀💀
2 : Grande Giorgia! Ti volevo anche chiedere: dici che noi giovani dovremo andare in pensione a 100 anni o per i 90 ci riusciamo?
3 : do you know applicare??
4 : Su questo ha perfettamente ragione e la 194 in pochi l'hanno letta e ne conoscono ogni suo punto .Giusto apicarla nella sua interezza
5 : aiuto
6 : lol
7 : bro
8 : love ❤️
9 : Più chiara di così nn può essere stata👏👏👏e chi nn lo capisce nn conosce l italiano🙈🙈🙈, Giorgia avanti nn ditroeggiare🥰🥰🥰🥰🥰🥰🥰🥰
10 : spero che se la storia si ripete veramente la marcia su roma sarà tipo un evento per la vincita dell’europa league. forza maggica❤️❤️
11 : Non ho capito
12 : quando ci aumentate le paghe le pensioni e vi diminuite i vostri 40.000,00€ al mese ,provate voi a vivere con 1500€ al mese benzina luce gas il cibo
13 : “do you know APPLICARE?”
14 : ma non capiranno mai
contestano tutto quello c